In [10]:
%load_ext autoreload
%autoreload 2
from convnet import ConvDipNet
from timeDistributed import TimeDistributed, TimeDistributedLinear
from torchinfo import summary
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch
import sys; sys.path.insert(0, '../')
from esinet.forward import create_forward_model, get_info
from esinet import Simulation
from copy import deepcopy
from CNN_LSTM.util import *
from dipoleDataset import DipoleDataset
import os
import mne
from cnn_lstm import CNNLSTM

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# define hyperparameters
in_channels = 1
im_shape = (9,9)
n_filters = 8
kernel_size = (3,3)

# create single input ConvDipNet
cnnlstm = CNNLSTM(in_channels, im_shape, n_filters, kernel_size)

# print model summary
summary(cnnlstm, input_size=(32, 100, 1, im_shape[0], im_shape[1])) # (batch_size, n_timesteps, in_channels, height, width)

torch.Size([32, 100, 5124])
torch.Size([32, 100, 512])
torch.Size([32, 100, 1024])


Layer (type:depth-idx)                   Output Shape              Param #
CNNLSTM                                  [32, 100, 5124]           --
├─TimeDistributed: 1-1                   [32, 100, 5124]           --
│    └─ConvDipNet: 2-1                   [3200, 5124]              --
│    │    └─Conv2d: 3-1                  [3200, 8, 9, 9]           80
│    │    └─BatchNorm2d: 3-2             [3200, 8, 9, 9]           16
│    │    └─Linear: 3-3                  [3200, 512]               332,288
│    │    └─BatchNorm1d: 3-4             [3200, 512]               1,024
│    │    └─Linear: 3-5                  [3200, 5124]              2,628,612
├─TimeDistributedLinear: 1-2             [32, 100, 512]            --
│    └─Linear: 2-2                       [3200, 512]               2,624,000
├─LSTM: 1-3                              [32, 100, 1024]           6,299,648
├─LayerNorm: 1-4                         [32, 100, 1024]           2,048
├─TimeDistributedLinear: 1-5             [32, 100, 51